# Proyecto Individual #1

> DATAFT08 - Jorge Marcelo Mendez

### Desarrollo API

Importo librerias necesarias, asi como tambien realizo la ingesta de dataframe requerida.

In [17]:
#Importo librerías
from fastapi import FastAPI, HTTPException
import pandas as pd
from pandasql import sqldf

In [18]:
#Cargo el dataset en una variable
df = pd.read_csv("streaming_completo.csv")

In [ ]:
#reviso que se haya cargado correctamente:
df

Ahora bien debo trabajar con los CSV's de la carpeta "ratings", ya que de ello debo sacar un valor de puntuacion promedio de cada pelicula por servicio de streaming.

---

In [19]:
# Leer archivos CSV
ratings_1_df = pd.read_csv(r"DATASETS_Henry\ratings\1.csv") 
ratings_2_df = pd.read_csv(r"DATASETS_Henry\ratings\2.csv")
ratings_3_df = pd.read_csv(r"DATASETS_Henry\ratings\3.csv")
ratings_4_df = pd.read_csv(r"DATASETS_Henry\ratings\4.csv")
ratings_5_df = pd.read_csv(r"DATASETS_Henry\ratings\5.csv")
ratings_6_df = pd.read_csv(r"DATASETS_Henry\ratings\6.csv")
ratings_7_df = pd.read_csv(r"DATASETS_Henry\ratings\7.csv")
ratings_8_df = pd.read_csv(r"DATASETS_Henry\ratings\8.csv")

# Concatenar dataframes
ratings_df = pd.concat([ratings_1_df, ratings_2_df, ratings_3_df, ratings_4_df, ratings_5_df, ratings_6_df, ratings_7_df, ratings_8_df])

In [20]:
# Calcular promedio de rating por movieId
ratings_df = ratings_df.groupby('movieId')['rating'].mean().reset_index()
ratings_df['rating'] = ratings_df['rating'].apply(lambda x: round(x, 1))

In [21]:
# Eliminar duplicados
ratings_df = ratings_df.drop_duplicates(subset='movieId')

Para poder relacionar los dataframes, debo realizar 2 acciones, la primera es eliminar la columna de show_id del dataframe de "df" y luego debo cambiar el nombre de la columna "movieId" por "id" y de esta manera hacer el merge de ambos dataframes y obtener uno consolidado con el nombre de "streaming_ratings_df".

In [22]:
df = df.drop(columns=['show_id'])

Reviso mis cambios

In [ ]:
print(df.columns)

In [ ]:
df.head()

In [23]:
ratings_df = ratings_df.rename(columns={'movieId': 'id'})

Reviso si los cambios fueron realizados en ambas base de datos, para luego hacer el merge y tener un solo dataframe consolidado

In [ ]:
print(ratings_df.columns)

In [ ]:
ratings_df.head()

In [24]:
# Unir dataframes
streaming_ratings_df = pd.merge(df, ratings_df, on='id')

In [25]:
print(streaming_ratings_df.columns)

Index(['type', 'title', 'director', 'cast', 'country', 'date_added',
       'release_year', 'rating_x', 'duration', 'listed_in', 'description',
       'platform', 'id', 'duration_int', 'duration_type', 'rating_y'],
      dtype='object')


In [26]:
streaming_ratings_df = streaming_ratings_df.rename(columns={'rating_x': 'rating'})
streaming_ratings_df = streaming_ratings_df.rename(columns={'rating_y': 'scored'})
streaming_ratings_df = streaming_ratings_df.rename(columns={'release_year': 'year'})

In [ ]:
print(streaming_ratings_df.columns)

In [ ]:
streaming_ratings_df

In [27]:
#exporto el archivo a un csv nuevo:
streaming_ratings_df.to_csv("streaming_ratings.csv", index=False)

HASTA ESTA ETAPA O PUNTO todas las pruebas han sido en mi entorno local 

---

Ahora, procedemos a cumplir con cada paso requerido en esta etapa de Desarrollo API

### PRIMERA CONSULTA

Película con mayor duración con filtros opcionales de AÑO, PLATAFORMA Y TIPO DE DURACIÓN. (la función debe llamarse get_max_duration(year, platform, duration_type))

In [ ]:
print(streaming_ratings_df.columns)

In [ ]:
streaming_ratings_df

In [38]:
from fastapi import FastAPI
import pandas as pd

app = FastAPI()

# Leer el archivo CSV con los datos de películas y series en streaming
streaming_ratings_df = pd.read_csv("streaming_ratings.csv")

@app.get("/max_duration")
async def get_max_duration(year: int = None, platform: str = None, duration_type: str = None):
    """
    Película con mayor duración con filtros opcionales de AÑO, PLATAFORMA Y TIPO DE DURACIÓN
    """
    # Crear una copia del dataframe para no modificar el original
    df_filtered = streaming_ratings_df.copy()
    
    # Filtrar por año
    if year is not None:
        df_filtered = df_filtered[df_filtered["year"] == year]
    else:
        return "Debes ingresar un año."
        
    # Filtrar por plataforma
    if platform is not None:
        df_filtered = df_filtered[df_filtered["platform"] == platform.lower()]
    else:
        return "Debes ingresar una plataforma."
 
    # Filtrar por tipo de duración
    if duration_type is not None:
        if duration_type.lower() == "min":
            df_filtered = df_filtered[df_filtered["duration_min"].notnull()]
            max_duration_col = "duration_min"
        elif duration_type.lower() == "season":
            df_filtered = df_filtered[df_filtered["duration_seasons"].notnull()]
            max_duration_col = "duration_seasons"
        else:
            return "Tipo de duración inválido."
    else:
        # Si no se especifica, se usa la duración más larga (entre minutos y temporadas)
        df_filtered = df_filtered[df_filtered["duration_min"].notnull() | df_filtered["duration_seasons"].notnull()]
        max_duration_col = "duration_min" if df_filtered["duration_min"].max() > df_filtered["duration_seasons"].max() else "duration_seasons"
    
    # Encontrar la película con la duración máxima
    max_duration_idx = df_filtered[max_duration_col].idxmax()
    max_duration_movie = df_filtered.loc[max_duration_idx, :]
    
    return f'La película con mayor duración de "{max_duration_movie["platform"]}" es "{max_duration_movie["title"]}" con {max_duration_movie[max_duration_col]} {duration_type or "minutos/temporadas"}.'

Reviso con un ejemplo:

In [39]:
get_max_duration(year=2010, platform='netflix', duration_type='season')

<coroutine object get_max_duration at 0x000002CE0D6C10E0>

### SEGUNDA CONSULTA

Cantidad de películas por plataforma con un puntaje mayor a XX en determinado año (la función debe llamarse get_score_count(platform, scored, year))

Primero debo generar un archivo completo, que incluya los scores promedio

In [213]:
print(streaming_ratings_df.columns)

Index(['type', 'title', 'director', 'cast', 'country', 'date_added', 'year',
       'rating', 'duration', 'listed_in', 'description', 'platform', 'id',
       'duration_int', 'duration_type', 'scored'],
      dtype='object')


In [32]:
@app.get('/score_count/')
def get_score_count(platform: str, scored: float, year: int = None):
    # Filtrar los datos según los valores de los parámetros
    filtered = streaming_ratings_df.copy()
    filtered = filtered[filtered['platform'] == platform]
    if year is not None:
        filtered = filtered[filtered['year'] == year]
    
    # Contar la cantidad de películas por plataforma y devolver los resultados
    count = len(filtered[filtered['scored'] > scored])
    return {'count': count}

In [33]:
#Reviso que la consulta funcione, con parametros de ejemplo:
get_score_count(platform='netflix',scored=3 , year=2009)

{'count': 152}

### TERCERA CONSULTA

Cantidad de películas por plataforma con filtro de PLATAFORMA. (La función debe llamarse get_count_platform(platform))

In [245]:
print(streaming_ratings_df.columns)

Index(['type', 'title', 'director', 'cast', 'country', 'date_added', 'year',
       'rating', 'duration', 'listed_in', 'description', 'platform', 'id',
       'duration_int', 'duration_type', 'scored'],
      dtype='object')


In [34]:
# Consulta 3: Cantidad de películas por plataforma y filtrar los resultados según la plataforma indicada
@app.get('/count_platform/')
def get_count_platform(platform: str):
    count = len(streaming_ratings_df[streaming_ratings_df['platform'] == platform])
    return {'count': count}

In [35]:
#Reviso que la consulta funcione, con parametros de ejemplo:
get_count_platform(platform='netflix')

{'count': 8807}

### CUARTA CONSULTA

Actor que más se repite según plataforma y año. (La función debe llamarse get_actor(platform, year))

In [248]:
print(streaming_ratings_df.columns)

Index(['type', 'title', 'director', 'cast', 'country', 'date_added', 'year',
       'rating', 'duration', 'listed_in', 'description', 'platform', 'id',
       'duration_int', 'duration_type', 'scored'],
      dtype='object')


In [36]:
@app.get('/actor/')
def get_actor(platform, year):
    # Filtrar por plataforma y año
    df_filtered = streaming_ratings_df[(streaming_ratings_df['platform'] == platform) & (streaming_ratings_df['year'] == year)]
    # Concatenar los valores de la columna "cast"
    cast_values = df_filtered['cast'].str.cat(sep=',').split(',')
    # Contar la frecuencia de cada actor
    actor_count = pd.Series(cast_values).value_counts()
    # Obtener el actor con mayor frecuencia
    most_common_actor = actor_count.idxmax()
    return most_common_actor

In [37]:
#Reviso que la consulta funcione, con parametros de ejemplo:
get_actor(platform='netflix', year='2009')

''